# **Bước 1:** Mount Drive và Khai báo các đường dẫn

In [ ]:
from google.colab import drive
import os, shutil

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

Mon Dec 22 19:32:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
DATASET_NAME = "carla_128"
ZIP_FILE = f"{DATASET_NAME}.zip"
DRIVE_PATH = f"/content/drive/MyDrive/HyperNeRFGAN/data/{ZIP_FILE}"
LOCAL_PATH = f"/content/{ZIP_FILE}"

# **Bước 2:** Copy dataset xuống Local (để tăng tốc độ xử lý)


In [ ]:
if os.path.exists(DRIVE_PATH):
    print(f"🔄 Đang copy {ZIP_FILE} từ Drive xuống Local...")
    shutil.copy(DRIVE_PATH, LOCAL_PATH)
    print(f"✅ Đã copy xong: {LOCAL_PATH}")
else:
    raise FileNotFoundError(f"❌ Không tìm thấy file: {DRIVE_PATH}. Hãy kiểm tra lại folder data trên Drive.")

🔄 Đang copy carla_128.zip từ Drive xuống Local...
✅ Đã copy xong: /content/carla_128.zip


# **Bước 3:** Cài đặt môi trường Micromamba

In [ ]:
%cd /content/drive/MyDrive/HyperNeRFGAN

# Cài Micromamba
!mkdir -p bin
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
!chmod +x bin/micromamba

# Tạo file environment tối ưu cho Colab
import yaml
env_in = "environment.yaml"
env_out = "environment_colab.yaml"

# Load environment gốc và lọc bỏ imageio-ffmpeg (cài pip sau)
if os.path.exists(env_in):
    y = yaml.safe_load(open(env_in, "r"))
    y["channels"] = ["conda-forge", "defaults"] # Bỏ pytorch channel để cài thủ công
    deps = []
    for d in y.get("dependencies", []):
        if isinstance(d, str) and (d.startswith("imageio-ffmpeg") or d.startswith("pytorch") or d.startswith("torchvision")):
            continue
        deps.append(d)
    y["dependencies"] = deps
    with open(env_out, "w") as f:
        yaml.safe_dump(y, f, sort_keys=False)
else:
    with open(env_out, "w") as f:
        f.write("channels:\n  - conda-forge\ndependencies:\n  - python=3.8\n  - pip\n")

print("✅ Đã tạo file config môi trường:", env_out)

# Tạo môi trường
ENV_PREFIX = "/content/hypernerfgan_env"
!rm -rf {ENV_PREFIX}
!./bin/micromamba create -y -p {ENV_PREFIX} -f environment_colab.yaml


/content/drive/MyDrive/HyperNeRFGAN
bin/micromamba
✅ Đã tạo file config môi trường: environment_colab.yaml
conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/
[+] 0.0s
[+] 0.1s
pkgs/main/noarch  ⣾  
pkgs/r/linux-64   ⣾  
pkgs/r/noarch     ⣾  

Transaction

  Prefix: /content/hypernerfgan_env

  Updating specs:

   - python=3.8.5
   - pip=20.3
   - cudatoolkit=10.2
   - imageio
   - protobuf=3.20.1
   - pip


  Package                Version  Build                 Channel           Size
────────────────────────────────────────────────────────────────────────────────
  Install:
────────────────────────────────────────────────────────────────────────────────


# **Bước 4:** Cài đặt Pytorch và Dependencies

In [ ]:
print("⬇️ Đang cài đặt PyTorch và thư viện...")

# 1. Cài PyTorch chuẩn (CUDA 11.3)
!./bin/micromamba run -p {ENV_PREFIX} \
  pip install torch==1.10.0+cu113 torchvision==0.11.0+cu113 \
  --extra-index-url https://download.pytorch.org/whl/cu113

# 2. Cài cố định phiên bản Pip và Setuptools cũ (Quan trọng)
# (Pip 23.3.1 để fix lỗi omegaconf, Setuptools<59.6 để fix lỗi distutils)
!./bin/micromamba run -p {ENV_PREFIX} pip install "pip==23.3.1" "setuptools<59.6.0" wheel

# 3. Cài repo ở chế độ editable
!./bin/micromamba run -p {ENV_PREFIX} pip install -e .

# 4. Cài Hydra & Dependencies
!./bin/micromamba run -p {ENV_PREFIX} \
  pip install hydra-core==1.0.6 omegaconf==2.0.6 antlr4-python3-runtime==4.8

# 5. Cài các util khác
!./bin/micromamba run -p {ENV_PREFIX} \
  pip install imageio-ffmpeg opencv-python tensorboard tqdm gpustat gitpython ninja

print("✅ Đã cài đặt xong môi trường (Dependencies OK).")


# # 1. Cài PyTorch chuẩn (CUDA 11.3)
# !./bin/micromamba run -p {ENV_PREFIX} \
#   pip install torch==1.10.0+cu113 torchvision==0.11.0+cu113 \
#   --extra-index-url https://download.pytorch.org/whl/cu113

# # 2. QUAN TRỌNG: Cài cố định phiên bản Pip và Setuptools cũ
# !./bin/micromamba run -p {ENV_PREFIX} pip install "pip==23.3.1" "setuptools<59.6.0" wheel

# # 3. Cài repo ở chế độ editable
# !./bin/micromamba run -p {ENV_PREFIX} pip install -e .

# # 4. Cài Hydra & Dependencies
# !./bin/micromamba run -p {ENV_PREFIX} \
#   pip install hydra-core==1.0.6 omegaconf==2.0.6 antlr4-python3-runtime==4.8

# # 5. Cài các util khác VÀ CHỐT NUMPY 1.23.5
# !./bin/micromamba run -p {ENV_PREFIX} \
#   pip install imageio-ffmpeg opencv-python tensorboard tqdm gpustat gitpython ninja "numpy==1.23.5"

# print("✅ Đã cài đặt xong môi trường (Dependencies OK).")


⬇️ Đang cài đặt PyTorch và thư viện...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached https://download.pytorch.org/whl/cu113/torch-1.10.0%2Bcu113-cp38-cp38-linux_x86_64.whl (1821.4 MB)
  Using cached https://download.pytorch.org/whl/cu113/torchvision-0.11.0%2Bcu113-cp38-cp38-linux_x86_64.whl (21.8 MB)
  Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Using cached setuptools-59.5.0-py3-none-any.whl (952 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.3.0
    Uninstalling setuptools-75.3.0:
      Successfully uninstalled setuptools-75.3.0
  Attempting uninstall: pip
    Found existing installation: pip 20.3.4
    Uninstalling pip-20.3.4:
      Successfully uninstalled pip-20.3.4
Obtaining file:///content/drive/MyDrive/HyperNeRFGAN
  Preparing metadata (setup.py) ... done
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour chang

# **Bước 5:** Patch config cho `main.yml`

In [ ]:
main_cfg = "configs/main.yml"
import re
if os.path.exists(main_cfg):
    txt = open(main_cfg, "r").read()
    txt = re.sub(r"^num_gpus:\s*\d+.*$", "num_gpus: 1", txt, flags=re.M)
    open(main_cfg, "w").write(txt)
    print("✅ Đã patch configs/main.yml")

✅ Đã patch configs/main.yml
✅ Đã giảm cấu hình memory trong configs/nerf-gan-carla.yml


# **Bước 6:** Memory fix ứng với từng file `<dataset>.yml`

In [ ]:
# Fix OOM cho T4 GPU
carla_cfg = "configs/nerf-gan-carla.yml"
if os.path.exists(carla_cfg):
    c = open(carla_cfg,"r").read()
    # Giảm từ 64 xuống 32 hoặc 48 để tránh tràn VRAM
    c = c.replace("patch_size: 64", "patch_size: 32")
    c = c.replace("n_samples: 64", "n_samples: 32")
    open(carla_cfg,"w").write(c)
    print(f"✅ Đã giảm cấu hình memory trong {carla_cfg}")

# **Bước 7:** Xử lý đồng loạt cái file `*.yml` chuẩn bị cho training

In [ ]:
import glob
import os
import re

CONFIG_DIR = "configs"
config_dir = "/content/drive/MyDrive/HyperNeRFGAN/configs"

# Lấy tất cả file trong folder configs
files = os.listdir(config_dir)

for f in files:
    full_path = os.path.join(config_dir, f)

    # Nếu là .yaml -> copy thêm bản .yml
    if f.endswith(".yaml"):
        target = full_path.replace(".yaml", ".yml")
        if not os.path.exists(target):
            shutil.copy(full_path, target)
            print(f"   Created clone: {os.path.basename(target)}")

    # Nếu là .yml -> copy thêm bản .yaml
    elif f.endswith(".yml"):
        target = full_path.replace(".yml", ".yaml")
        if not os.path.exists(target):
            shutil.copy(full_path, target)
            print(f"   Created clone: {os.path.basename(target)}")

print("✅ Configs đã sẵn sàng (Dual extension).")

# 2. Patch file main.yaml (Set 1 GPU)
main_cfg = os.path.join(CONFIG_DIR, "main.yaml")
if os.path.exists(main_cfg):
    txt = open(main_cfg, "r").read()
    # Force 1 GPU
    txt = re.sub(r"^num_gpus:\s*\d+.*$", "num_gpus: 1", txt, flags=re.M)
    # Sửa tham chiếu đến file config con (cũng phải đổi thành yaml nếu có hardcode)
    open(main_cfg, "w").write(txt)
    print("✅ Đã patch main.yaml (num_gpus=1)")

# 3. Patch file dataset config (Fix OOM cho T4 GPU)
# Tự động tìm file config tương ứng với DATASET_NAME
target_cfg_name = f"nerf-gan-{DATASET_NAME.split('_')[0]}" # Vd: carla_128 -> nerf-gan-carla
dataset_cfg = os.path.join(CONFIG_DIR, f"{target_cfg_name}.yaml")

if os.path.exists(dataset_cfg):
    c = open(dataset_cfg, "r").read()
    # Giảm thông số để vừa với 16GB VRAM của T4
    c = c.replace("patch_size: 64", "patch_size: 32")
    c = c.replace("n_samples: 64", "n_samples: 32") # Giảm bớt số mẫu trên tia
    open(dataset_cfg, "w").write(c)
    print(f"✅ Đã tối ưu bộ nhớ trong {dataset_cfg}")
else:
    print(f"⚠️ Cảnh báo: Không tìm thấy config {dataset_cfg}. Có thể tên dataset khác với quy tắc đặt tên config.")

✅ Configs đã sẵn sàng (Dual extension).
✅ Đã patch main.yaml (num_gpus=1)
✅ Đã tối ưu bộ nhớ trong configs/nerf-gan-carla.yaml


# **Bước 8:** Chạy training

In [ ]:
print("🚀 Bắt đầu Training...")

!./bin/micromamba run -p {ENV_PREFIX} \
  python src/infra/launch_local.py hydra.run.dir=. \
  +experiment_name=carla_128_t4_run1 \
  +dataset.name=carla_128 \
  +dataset.root_path={LOCAL_PATH} \
  num_gpus=1 \
  +D_kwargs.patch_size=32 \
  +G_kwargs.synthesis_kwargs.patch_size=32 \
  +G_kwargs.cfg.patch_size=32

🚀 Bắt đầu Training...
<=== CONFIG ===>
env:
  python_bin: python
  base_project_dir: ${hydra:runtime.cwd}
  before_train_commands: []
  torch_extensions_dir: /tmp/torch_extensions
  datasets_dir: data
  objects_to_copy:
  - ${env.base_project_dir}/src
  - ${env.base_project_dir}/configs
  symlinks_to_create:
  - ${env.base_project_dir}/data
num_gpus: 1
dataset:
  source_path: ${env.datasets_dir}/${dataset.name}.zip
  target_path: data/${dataset.name}.zip
  name: carla_128
  root_path: /content/carla_128.zip
print_only: false
project_release_dir: /content/drive/MyDrive/HyperNeRFGAN/experiments/carla_128_t4_run1-None
train_args:
  outdir: ${project_release_dir}
  data: ${dataset.target_path}
  gpus: ${num_gpus}
  metrics: fid1k_full
  cfg: nerf
  generate_video: true
  video_front_view: true
  snap: 50
  aug: fixed
  augpipe: crop
  p: 0.5
  mirror: 1
  hydra_cfg_name: nerf-gan-carla.yml
train_args_str: --outdir=/content/drive/MyDrive/HyperNeRFGAN/experiments/carla_128_t4_run1-None
  --d